<h1>Pearsons R Recommeder built with Python and Pandas</h1>

In [46]:
import pandas as pd
import numpy as np

<p>The dataset I've used is a popular MovieLens dataset available for public download <a href="https://grouplens.org/datasets/movielens/">here</a>.</p>
<p><b>Dataset description:</b> 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.</p>

In [47]:
movies = pd.read_csv('ml-latest-small/movies.csv')
print(f'movies dataframe has {movies.shape[0]} rows and {movies.shape[1]} columns')
movies.tail(5)

movies dataframe has 9742 rows and 3 columns


,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [48]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
print(f'ratings dataframe has {ratings.shape[0]} rows and {ratings.shape[1]} columns')
ratings.head()

ratings dataframe has 100836 rows and 4 columns


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<p>Will be convinient to replace movieId with movie title in the rating column. To do that we may use .map mathod that is <i>used for substituting each value in a Series with another value, that may be derived from a function, a dict or a Series.</i></p>


In [49]:
%%time
def replace_id_with_name(id):
    return movies[movies['movieId'] == id].title.values[0]

ratings.movieId = ratings.movieId.map(replace_id_with_name)

CPU times: user 52.8 s, sys: 11.6 ms, total: 52.8 s
Wall time: 53 s


In [50]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,Toy Story (1995),4.0,964982703
1,1,Grumpier Old Men (1995),4.0,964981247
2,1,Heat (1995),4.0,964982224
3,1,Seven (a.k.a. Se7en) (1995),5.0,964983815
4,1,"Usual Suspects, The (1995)",5.0,964982931


<p>To create recommendations we have to build s pivot table using <i>pivot_table</i> command.</p>


In [86]:
matrix = ratings.pivot_table(index=['userId'],
                             columns=['movieId'],
                             values='rating')
print(f"""matrix_of_users_by_movies has {matrix.shape[0]} rows and {matrix.shape[1]} columns""")

matrix.head(20)

matrix_of_users_by_movies has 610 rows and 9719 columns


movieId,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


<p>This table is very sparse, it contains mostly NaN values which is kind of natural because not every person saw every movie.</p>


In [93]:
def pearson_r(series1, series2):
    """Calculate correlation between two series"""
    series1_c = series1 - series1.mean()
    series2_c = series2 - series2.mean()
    pearson_r_value = np.sum(series1_c * series2_c) / np.sqrt(np.sum(series1_c ** 2) * np.sum(series2_c ** 2))
    return {'pearson_r_value': pearson_r_value,
            'series1_non_Nan': series1.count(),
            'series2_non_Nan': series2.count(),
            'np.sum(series1_c * series2_c)': np.sum(series1_c * series2_c),
            'non-NaN_products': (series1_c * series2_c).count()}

<p>For two random movies whick does not seem to be correlated in any way the correlation coefficient is as follows:</p>

In [94]:
for k, v in pearson_r(matrix['Sherlock - A Study in Pink (2010)'], 
                      matrix['(500) Days of Summer (2009)']).items():
    print(k, '-->', v)

pearson_r_value --> 0.05222329678670935
series1_non_Nan --> 2
series2_non_Nan --> 42
np.sum(series1_c * series2_c) --> 0.125
non-NaN_products --> 2


<p>But is it really a reliable metrics given the fact that we calculated it based only on two rows with non-NaN values simultaneously?</p>

<p>What about the movie Matrix? At first we should find the exact names of this movies.</p>


In [95]:
movies[movies['title'].str.match('Matr')]

,movieId,title,genres
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4351,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4639,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


In [105]:
pearson_r(matrix['Matrix Reloaded, The (2003)'], 
          matrix['Matrix Revolutions, The (2003)'])

{'pearson_r_value': 0.5492530881058778,
 'series1_non_Nan': 96,
 'series2_non_Nan': 79,
 'np.sum(series1_c * series2_c)': 54.24459388185654,
 'non-NaN_products': 64}

<p>Now the coefficient is a lot more reliable because we have 64 non-NaN products and 0.54 indicates that movies are pretty close to each other.</p>


<p>What about Harry Potter? All movies from the series should be close to each other.</p>


In [111]:
movies[movies['title'].str.match('Harry')]

,movieId,title,genres
2528,3388,Harry and the Hendersons (1987),Children|Comedy
3574,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy
4076,5816,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
5166,8368,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy|IMAX
6062,40815,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller|IMAX
6522,54001,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy|IMAX
7078,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX
7285,74948,Harry Brown (2009),Crime|Drama|Thriller
7465,81834,Harry Potter and the Deathly Hallows: Part 1 (...,Action|Adventure|Fantasy|IMAX
7644,88125,Harry Potter and the Deathly Hallows: Part 2 (...,Action|Adventure|Drama|Fantasy|Mystery|IMAX


In [112]:
pearson_r(matrix['Harry Potter and the Chamber of Secrets (2002)'], 
          matrix['Harry Potter and the Prisoner of Azkaban (2004)'])

{'pearson_r_value': 0.42701539417752366,
 'series1_non_Nan': 102,
 'series2_non_Nan': 93,
 'np.sum(series1_c * series2_c)': 27.891787897954877,
 'non-NaN_products': 76}

In [113]:
pearson_r(matrix['Harry Potter and the Goblet of Fire (2005)'], matrix['Matrix, The (1999)'])

{'pearson_r_value': 0.052211472313611715,
 'series1_non_Nan': 71,
 'series2_non_Nan': 278,
 'np.sum(series1_c * series2_c)': 4.98632080251292,
 'non-NaN_products': 55}

In [114]:
pearson_r(matrix['Harry Potter and the Goblet of Fire (2005)'], matrix['Harry Potter and the Prisoner of Azkaban (2004)'])

{'pearson_r_value': 0.4357134402128312,
 'series1_non_Nan': 71,
 'series2_non_Nan': 93,
 'np.sum(series1_c * series2_c)': 18.002726033621084,
 'non-NaN_products': 61}